<a href="https://colab.research.google.com/github/Cauch-BS/VaxPress/blob/main/tests%20(jupyter)/VaxPressTestEnv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
from time import time
import numpy as np
from Bio.Data import CodonTable
from collections import namedtuple
from typing import Callable
from itertools import cycle
import pandas as pd
import numpy as np

STOP = "*"
MutationChoice = namedtuple('MutationChoice', ['pos', 'altcodon'])

class Sequence:
    def __init__(self, cdsseq: str, codon_table: str = 'standard', is_protein: bool = False):
        self.initialize_codon_table(codon_table)

        if is_protein:
            self.cdsseq = self.backtranslate(cdsseq)
        else:
            if type(cdsseq) == str:
                self.cdsseq = cdsseq.upper()
            elif type(cdsseq) == list and type(cdsseq[0]) == str:
                self.cdsseq = ''.join(cdsseq)


        if len(self.cdsseq) % 3 != 0:
            raise ValueError("Invalid CDS sequence length!")

        self.aaseq = self.translate(self.cdsseq)
        self.codons = [self.cdsseq[i:i+3] for i in range(0, len(self.cdsseq), 3)]

    def initialize_codon_table(self, codon_table: str) -> None:
        table_var_name = f'{codon_table}_rna_table'
        if not hasattr(CodonTable, table_var_name):
            raise ValueError(f'Invalid codon table name: {codon_table}')

        self.codon_table = getattr(CodonTable, table_var_name)

        codon_frame = pd.DataFrame(
            list(self.codon_table.forward_table.items()) +
            [[stopcodon, STOP] for stopcodon in self.codon_table.stop_codons],
            columns = ['codon', 'aa']
        )

        self.synonymous_codons, self.aa2codons, self.codon2aa = {}, {}, {}

        for aa, codons in codon_frame.groupby('aa'):
            codons = set(codons['codon'])
            self.aa2codons[aa] = codons

            for codon in codons:
                self.synonymous_codons[codon] = sorted(codons - set([codon]))
                self.codon2aa[codon] = aa

    def backtranslate(self, proteinseq: str) -> str:
        return ''.join(next(iter(self.aa2codons[aa])) for aa in proteinseq)

    def translate(self, rnaseq: str) -> str:
        return ''.join(self.codon2aa[rnaseq[i:i+3]] for i in range(0, len(rnaseq), 3))

class MutantGenerator(Sequence):
    cdsseq = None
    initial_codons = None

    def __init__(self, cdsseq: str, codon_table: str = 'standard', is_protein: bool = False,
                 random_state: np.random.RandomState = None):
        super().__init__(cdsseq, codon_table, is_protein)
        self.rand = np.random if random_state is None else random_state
        self.initial_codons = self.codons[:]
        self.setup_choices()

    def setup_choices(self) -> None:
        choices = []

        for i in range(len(self.cdsseq) // 3):
            codon = self.cdsseq[i*3 : i*3 + 3]
            alternatives = len(self.synonymous_codons[codon])
            for alt in range(alternatives):
                choices.append(MutationChoice(i, alt))

        self.choices = choices

    def randomize_initial_codons(self) -> None:
        self.initial_codons[:] = [
            self.rand.choice([codon] + self.synonymous_codons[codon])
            for codon in self.initial_codons]

    def generate_mutant(self, mutation_rate: float, codons: list[str] = None,
                        choices: list[MutationChoice] = None) -> list[str]:

        child = codons[:] if codons is not None else self.initial_codons[:]

        if choices is None:
            choices = self.choices

        # Draw Number of Mutations from the Binomial Distribution
        n_mutations = self.rand.binomial(len(choices), mutation_rate)
        n_mutations = max(1, min(n_mutations, len(choices)))

        #Selection of mutations
        mutation_choices = self.rand.choice(len(choices), n_mutations, replace = False)


        #Apply mutations
        for i in mutation_choices:
            mut = choices[i]

            child[mut.pos] = self.synonymous_codons[child[mut.pos]][mut.altcodon]

        self.codons = child
        self.cdsseq = ''.join(child)

        return child

    def compute_expected_mutations(self, mutation_rate: float) -> float:
        return len(self. choices) * mutation_rate

    def compute_mutational_space(self) -> dict:
        log10_totalcases = np.sum(np.log10(list((map(len, self.choices)))))
        totalcases_mantissa = 10 ** (log10_totalcases % 1)
        totalcases = f'{totalcases_mantissa:.2f} ✕ 10^{int(log10_totalcases)}'

        return {
            'singles' : len(self.choices),
            'total': totalcases,
        }

In [25]:
class BinaryCrossOver:
    def __init__(self, parent1: list[str] , parent2: list[str],
                 random_state: np.random.RandomState = None):
        """Initializes a recombination object with a list of sequences of codons"""
        self.sequence1 = parent1[:]
        self.sequence2 = parent2[:]

        assert len(self.sequence1) == len(self.sequence2), "Recombined Sequences must have the same length"

        self.length = len(self.sequence1)
        self.rand = np.random if random_state is None else random_state

    def single_point_crossover(self, crossover_prob: float):
        self.child1, self.child2 = self.sequence1[:], self.sequence2[:]
        pos_crossover = min(1, max(self.rand.binomial(self.length, crossover_prob), self.length - 1))
        #troubleshooting code
        #print(f"Crossver position: {pos_crossover}")
        for i in range(pos_crossover):
            self.child1[i], self.child2[i] = self.sequence2[i], self.sequence1[i]
        return self.child1, self.child2

    def two_point_crossover(self, crossover_prob: float):
        self.child1, self.child2 = self.sequence1[:], self.sequence2[:]
        start = self.rand.randint(0, self.length)
        diff = min(1, max(self.rand.binomial(self.length, crossover_prob), self.length - 1))
        if start + diff < self.length:
            end = start + diff
        elif start + diff >= self.length:
            if start > diff:
                start, end = start - diff, start
            else:
                if start < self.length // 2:
                    start, end = 0, start
                elif start >= self.length // 2:
                    start, end = start, self.length - 1
                else:
                    raise ValueError("Invalid start and end points")
        #troubleshooting code
        #print(f"Crossver position: {start} and {end}")
        for i in range(start, end):
            self.child1[i], self.child2[i] = self.sequence2[i], self.sequence1[i]

        return self.child1, self.child2

    def binomial_crossover(self, crossover_prob: float):
        self.child1, self.child2 = self.sequence1[:], self.sequence2[:]
        n_crossover = self.rand.binomial(self.length, crossover_prob)
        n_crossover = max(1, min(n_crossover, self.length))

        crossover_choices = self.rand.choice(self.length, n_crossover, replace = False)
        #troubleshooting code
        #print(f"Crossver positions: {crossover_choices}")
        for i in crossover_choices:
            self.child1[i], self.child2[i] = self.sequence2[i], self.sequence1[i]

        return self.child1, self.child2


In [27]:
if __name__ == "__main__":
    #Run Tests for Binary Crossover
    print("Running Tests for Binary Crossover")
    parent1 = ['GGG']*5000
    parent2 = ['AAA']*5000
    crossover = BinaryCrossOver(parent1, parent2)
    current_time = time()
    for tests in range(1000):
        single_point = crossover.single_point_crossover(0.2)
    print(time() - current_time)
    current_time = time()
    for tests in range(1000):
        two_point = crossover.two_point_crossover(0.2)
    print(time() - current_time)
    current_time = time()
    for tests in range(1000):
        binomial_point = crossover.binomial_crossover(0.2)
    print(time() - current_time)

    print("Done")

Running Tests for Binary Crossover
0.04765176773071289
0.049581289291381836
0.4378340244293213
Done


In [84]:
class Reproduction:
    def __init__(self, sequences_of_parents: list[list], n_offspring: int = 100,
                 method: str = "genetic_algorithm",
                 mutation_rate: float = 0.10, with_crossover: bool = True, crossover_method: str = 'binomial', crossover_prob: float = 0.10,
                 dif_cross_prob: float = 0.90, dif_scaling_factor: float = 2.0):
        self.parents = [Sequence(seq).codons for seq in sequences_of_parents]
        self.n_parents = len(self.parents)
        self.n_offspring = n_offspring
        self.mutation_rate = mutation_rate
        self.method = method
        self.offspring = []
        self.population = self.parents[:]
        self.population_size = len(self.parents)

        if self.method not in ['genetic_algorithm', 'differential_evolution', 'discrete_cmaes']:
            raise ValueError("Invalid method")
        if self.method == 'differential_evolution' or self.method == 'discrete_cmaes':
            raise NotImplementedError("Differential Evolution and Discrete CMAES are not yet implemented")
        else:
            self.has_crossover = with_crossover
            if self.has_crossover:
                self.crossover_method = crossover_method
                if crossover_method not in ['single_point', 'two_point', 'binomial']:
                    raise ValueError("Invalid crossover method")
                assert 0 < crossover_prob < 1, "Crossover probability must be between 0 and 1"
                self.crossover_prob = crossover_prob

    def generate_offspring(self):
        if self.method == 'genetic_algorithm':
            offspring = self.genetic_algorithm()
        elif self.method == 'differential_evolution' or self.method == 'discrete_cmaes':
            raise NotImplementedError("Differential Evolution and Discrete CMAES are not yet implemented")
        else:
            raise ValueError("Invalid method")
        return offspring

    def genetic_algorithm(self):
        offspring = []

        for i in range(self.n_offspring):
            parents = self.parents[:]
            parent1 = parents.pop(i % self.n_parents)
            #for troubleshooting
            #print(f"Parent 1: {parent1[:10]}")
            #print(f"Remaining population has length: {len(parents)}")

            if self.has_crossover:
                parent2idx = np.random.choice(range(self.n_parents - 1))
                parent2 = parents[parent2idx]
                #Select two parent randomly
                crossover = BinaryCrossOver(parent1, parent2)
                #Perform crossover
                if self.crossover_method == 'single_point':
                    child_codons, _ = crossover.single_point_crossover(self.crossover_prob)
                elif self.crossover_method == 'two_point':
                    child_codons, _ = crossover.two_point_crossover(self.crossover_prob)
                elif self.crossover_method == 'binomial':
                    child_codons, _ = crossover.binomial_crossover(self.crossover_prob)

            child_codons = parent1[:]

            #perform mutation
            child_codons = MutantGenerator(child_codons).generate_mutant(self.mutation_rate)
            offspring.append(child_codons)

        self.offspring = offspring
        self.population += offspring
        self.population_size = len(self.population)

        return offspring


In [85]:
if __name__ == "__main__":
    #Run Tests for Reproduction
    population = []
    print("Generating Parents")
    for i in range(100):
        base_char = ['A', 'U', 'G', 'C']
        bases = np.random.choice(base_char, 3000)
        population.append(bases)
        if i < 5:
            print(f"Parent {i} generated as {population[i][:10].tolist() + ['...']}")
        elif i == 6:
            print(f"continuing generation...")
        else:
            continue
    assert len(population) == 100, "Population must be of size 100"
    assert len(population[0]) == 3000, "Sequences mut be of length 3000"
    print("Parents generated successfully, Testing Reproduction...")

    joining = lambda arr: ''.join(arr)
    population = list(map(joining, population))

    parents = Reproduction(population)
    offspring = parents.generate_offspring()

    for i in range(5):
        print(f"Offspring {i} generated as {offspring[i][:10] + ['...']}")

    assert len(offspring) == 100, "Offspring must be of size 100"
    assert len(offspring[0]) == 1000, "Offspring must be of length 1000"
    print(f"Offspring generated successfully, {len(offspring)} offspring generated")



Generating Parents
Parent 0 generated as ['C', 'G', 'U', 'C', 'U', 'G', 'A', 'G', 'G', 'U', '...']
Parent 1 generated as ['C', 'C', 'G', 'U', 'C', 'A', 'U', 'A', 'U', 'A', '...']
Parent 2 generated as ['U', 'G', 'G', 'C', 'C', 'C', 'C', 'G', 'A', 'G', '...']
Parent 3 generated as ['G', 'U', 'U', 'C', 'C', 'C', 'A', 'G', 'A', 'C', '...']
Parent 4 generated as ['A', 'A', 'A', 'A', 'U', 'G', 'C', 'A', 'U', 'U', '...']
continuing generation...
Parents generated successfully, Testing Reproduction...
Offspring 0 generated as ['CGU', 'CUG', 'AGG', 'UGA', 'GUG', 'UUU', 'UCA', 'UGA', 'GAC', 'GGG', '...']
Offspring 1 generated as ['CCG', 'UCA', 'UAU', 'AAU', 'CUA', 'UGU', 'AGU', 'CAU', 'AUU', 'GAA', '...']
Offspring 2 generated as ['UGG', 'CCC', 'CGA', 'GCG', 'AAC', 'CGC', 'CAC', 'CUA', 'GCU', 'AUG', '...']
Offspring 3 generated as ['GUU', 'CCC', 'CGA', 'CCC', 'AGG', 'UGC', 'GGG', 'CUA', 'GGC', 'AGU', '...']
Offspring 4 generated as ['AAA', 'AUG', 'CAU', 'UAA', 'GAU', 'AGU', 'GUC', 'UCU', 'CCA',